<a href="https://www.kaggle.com/code/yamarieli/quantitative-ordinal-categorical-houses-prediction?scriptVersionId=123851512" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## The goal is to answer the following questions
<div>
    <span><b>Get familiar with the data</b></span>
    <ul>
        <li>What the type of each column?
        <ol>
            <li>quantitative</li>
            <li>ordinal</li>
            <li>categorical</li>
            </ol>
        </li>
        <li>What the meaning of each column?</li>
        <li>What are the values of each column?</li>
    </ul>
</div>

<div>
    <span><b>Clean the data</b></span>
    <ul>
        <li>What are the outlairs?</li>
        <li>how much should we clean?</li>
    </ul>
</div>

<div>
    <span><b>Prediction</b></span>
    <ul>
        <li>What is the best method for each type?</li>
        <li>How to apply the insights to the data?</li>
    </ul>
</div>

### Note:
**`In this notebook there are 3D animations, it is recommended to go down to see😃👌`**

In [ ]:
from math import ceil
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
from typing import Tuple, Union, List
import imageio
from IPython.display import Image
from scipy.stats import spearmanr

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df_test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
df

In [ ]:
df_xy = pd.read_excel('/kaggle/input/nei-xy-full/nei_full.xlsx')
df_xy.rename(columns={'short': 'Neighborhood'}, inplace=True)

In [ ]:
df_xy = df_xy.merge(right=df[['Neighborhood', 'SalePrice']], on='Neighborhood')

In [ ]:
import pandas as pd
import folium
from folium.plugins import HeatMap
from IPython.display import IFrame

In [ ]:
file_name = 'nei_heat.html'
tiles = 'openstreetmap' # 'cartodbpositron' is also available
location=[df_xy.Latitude.mean(), df_xy.Longitude.mean()]

this_map = folium.Map(location=location, tiles=tiles, zoom_start=12)
HeatMap(data=df_xy[['Latitude', 'Longitude']], radius=15).add_to(this_map)
this_map.save(file_name)

frame = IFrame(file_name, width='70%', height='500px')
frame

## Let's get familiar with the data

In [ ]:
df.columns

### wow! this is a lot...
### Let's understand the names meaning by 'data_description.txt' file
*(uncomment to see description)*

In [ ]:
with open('/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt', 'r') as f:
    print(f.read())

## Let's split by 3 types:
* quantitative
* ordinal
* categorical

## Continuous data:

In [ ]:
continuous_columns = {'GarageArea', 'SalePrice', 'GrLivArea', 'LowQualFinSF', '2ndFlrSF', '1stFlrSF', 'TotalBsmtSF', 'BsmtUnfSF', 'BsmtFinSF2', 'BsmtFinSF1', 'LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea'}
df_continuous = df[[*continuous_columns]]

### Take a first look at the columns

In [ ]:
continuous_columns.remove('SalePrice')
GRAPH_COUNT = len(continuous_columns)
GRAPH_IN_A_ROW = 2

fig, axs = plt.subplots(int(GRAPH_COUNT/GRAPH_IN_A_ROW), GRAPH_IN_A_ROW, figsize = (20, 25))

for col, ax in zip(continuous_columns, axs.flat):
    ax.set_title(col)
    ax.scatter(df_continuous[col].tolist(), df_continuous['SalePrice'].tolist())
plt.show()

### It's seems that the main correlation is with ['GrLivArea', '1stFlrSF', 'LotArea']
let's see it in a heatmap

In [ ]:
corr = df_continuous.corr()
fig, ax = plt.subplots(figsize = (20, 15))
sns.heatmap(corr, annot=True)

### It's seems that in the heat table you can see things differently
most of the columns that correlated with **SalePrice** are correlated with each other, so they wouldn't contribute to the prediction.<br>
Prediction by too much variables may be problematic:
<ul>
    <li>Each variable contributes a little noise. Many small noises add up to a big noise.</li>
    <li>Clearing a few outlairs from many variables may sum up to clearing many values and losing a large part of the learning data</li>
<ul>

So we should choose the best one and remove the rest, it will be **GrLivArea**.

Note that **YearBuilt** also in good correlation **SalePrice**, but unlike most of the other columns, it has a relatively low correlation with **GrLivArea**.<br>
Therefore it might help.

In [ ]:
cols = ['GrLivArea', 'YearBuilt']
df_continuous = df[['SalePrice', 'Neighborhood', *cols]].copy()

COLS_COUNT = len(cols)

### ML already?? not yet -> cleaning time

<h2>Is <b>3M$</b> is a lot?</h2>
    <h5>well, It depends...</h5>
<p>
    <span>
    The real question is whether <b>this price</b> represents a reasonable value for <b>this property</b>.<br>
    We will refer to:
    </span>
    <ul>
        <li>The <b>neighborhood</b> where the property is</li>
        <li>The main <b>continuous values</b></li>
    </ul>
    <span>Therefore I will:</span>
    <ol>
        <li>Calculate the ratio between the <b>price</b> and the <b>continuous values</b>  for each <b>neighborhood</b></li>
        <li>Remove the outlairs</li>
    </ol>
</p>

In [ ]:
def get_ratio(df: pd.DataFrame, col_name: str) -> Tuple[pd.DataFrame, str]:
    new_col_name = f'{col_name}_ratio'
    df[new_col_name] = df.apply(lambda row: row[col_name]/row['SalePrice'], axis=1).copy()
    return df, new_col_name

In [ ]:
def get_quantile_by_group(df: pd.DataFrame, groupby_column: str, quan_columns: List[str] , q_size=0.01) -> pd.DataFrame:
    grouped = df.groupby(groupby_column)
    
    # Compute the lower quantile for each group
    low_q_df = grouped[quan_columns].quantile(q_size)
    # Fix the the names of the columns
    low_q_df.rename(columns={col: f'{col}_low_quantile' for col in low_q_df.columns}, inplace=True)
    
    # Compute the higher quantile for each group
    high_q_df = grouped[quan_columns].quantile(1-q_size)
    # Fix the the names of the columns
    high_q_df.rename(columns={col: f'{col}_high_quantile' for col in high_q_df.columns}, inplace=True)

    return pd.merge(low_q_df, high_q_df, left_index=True, right_index=True)


In [ ]:
ratio_cols = []
for col in cols:
    df_continuous, ratio_col = get_ratio(df_continuous, col)
    ratio_cols.append(ratio_col)

print('ratio_cols = ', ratio_cols)
df_continuous.head()

In [ ]:
# Create Dataframe of high and low quantile of the columns we choose for each Neighborhood

df_Neighborhood_ratio_quantile = get_quantile_by_group(df_continuous, 'Neighborhood', ratio_cols)
df_Neighborhood_ratio_quantile.head()

In [ ]:
# Add the quantile columns to the df_continuous

df_continuous = pd.merge(left=df_continuous, right=df_Neighborhood_ratio_quantile,
         left_on='Neighborhood', right_index=True)
df_continuous.head()

In [ ]:
# Remove outlairs

for rc in ratio_cols:
    if df_continuous[rc].shape[0] > 4:
        df_continuous = df_continuous[
            (df_continuous[rc] >= df_continuous[f'{rc}_low_quantile']) &
            (df_continuous[rc] <= df_continuous[f'{rc}_high_quantile'])
        ]

In [ ]:
plt.rcParams["figure.figsize"] = (18,5)

for col in cols:
    plt.title(col)
    plt.scatter(df_continuous[col].tolist(), df_continuous['SalePrice'].tolist())
    plt.show()

In [ ]:
xdata, ydata, zdata = df_continuous['GrLivArea'], df_continuous['YearBuilt'], df_continuous['SalePrice']

horizontal_range = list(range(10, 70, 2))
vertical_range = horizontal_range

fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(211,projection='3d')
ax.scatter3D(xdata, ydata, zdata, c=zdata,  
                cmap='viridis', edgecolor='none');

ax.set_xlabel('GrLivArea')
ax.set_ylabel('YearBuilt')
ax.set_zlabel('SalePrice');

for i in horizontal_range:
    ax.view_init(i, i)
    plt.savefig(f'{i}.png')
    
plt.clf()

with imageio.get_writer('area_year_price.gif', mode='I', duration=0.15) as writer:
    for i in horizontal_range:
        file_name = f'{i}.png'
        image = imageio.imread(file_name)
        os.remove(file_name)
        writer.append_data(image)

with open('area_year_price.gif','rb') as f:
    display(Image(data=f.read(), format='png'))

<h2>
Looking much better!<br>
Now it's time for reggresion
</h2>

In [ ]:
df_continuous.groupby('Neighborhood')['Neighborhood'].count()

It's seems that some neighborhoods don't have a lot of data, so I will use all the data to train, and test a set of values in the range.

In [ ]:
def get_N_numbers_in_range(N: int, low: float, high: float):
    d = (high-low) / N
    return list(map(lambda x: (x*d)+(low-d), range(1, N+1)))

### Compute the regressor of each neighborhood
* The reggresor will be saved in the `dict` **area_year_regressors**
* The data for the plot will be saved in the `list` **fig_data**

In [ ]:
# this dict keys will be the neighborhoods names and the values will be the corresponding models
area_year_regressors = {}
fig_data = []

for nei in df_continuous.Neighborhood.unique():
    # only rows of this Neighborhood
    df_temp = df_continuous[df_continuous.Neighborhood == nei]
    X_train, y_train = df_temp[cols].to_numpy(), df_temp.SalePrice.to_numpy()
    
    poly = PolynomialFeatures(2)
    regressor = Ridge(alpha=1000)
    
    X_train_trans = poly.fit_transform(X_train)
    regressor.fit(X_train_trans, y_train)
    
    # add the new calculated regressor to the regressors dict
    area_year_regressors[nei] = regressor

    GLA_range = get_N_numbers_in_range(10, df_temp.GrLivArea.min(), df_temp.GrLivArea.max())
    YB_range = get_N_numbers_in_range(10, df_temp.YearBuilt.min(), df_temp.YearBuilt.max())
    
    X_test = np.array([[g, y] for g in GLA_range for y in YB_range])
    X_test_trans = poly.fit_transform(X_test)
    y_predict = regressor.predict(X_test_trans)

    xdata, ydata, zdata = df_temp['GrLivArea'], df_temp['YearBuilt'], df_temp['SalePrice']
    xtest, ytest, ztest = X_test[:,0], X_test[:,1], y_predict
    
    ax_data = {
        'nei': nei,
        'org_data': (xdata, ydata, zdata),
        'test': (xtest, ytest, ztest)
    }
    
    fig_data.append(ax_data)

#### For each angle I will create one big plot for all the subplots 
Save it as diffrent image

In [ ]:
single_sub_size = 10
ncols = 2
nrows = ceil(df_continuous.Neighborhood.unique().size/ncols)

fig = plt.figure(figsize=(single_sub_size*ncols, single_sub_size*nrows))
angles = list(range(10, 70, 2))

for angle in angles:
    for index, ax_data in enumerate(fig_data, start=1):    
        # create the fig object
        ax = fig.add_subplot(nrows, ncols, index, projection='3d')

        nei = ax_data['nei']
        org_data = ax_data['org_data']
        test = ax_data['test']

        ax.scatter3D(*org_data, c=org_data[2], cmap='viridis', edgecolor='none');
        ax.scatter3D(*test, c=test[2], cmap='Greens', edgecolor='none')

        ax.set_xlabel('GrLivArea')
        ax.set_ylabel('YearBuilt')
        ax.set_zlabel('SalePrice')
        ax.set_title(nei)
        ax.view_init(angle, angle)
    
    plt.savefig(f'Neighborhoods{angle}.png')
    plt.clf()
    

#### build a gif from all the images

In [ ]:
with imageio.v2.get_writer(f'Neighborhoods.gif', mode='I', duration=0.15) as writer:
    for angle in angles:
        file_name = f'Neighborhoods{angle}.png'
        image = imageio.v2.imread(file_name)
        writer.append_data(image)
        
        # delete the image
        os.remove(file_name)

In [ ]:
with open('Neighborhoods.gif','rb') as f:
    display(Image(data=f.read(), format='png'))

### The prediction in green is looking good
It seems that the selection of the variables for the regression was correct,<br>
there are neighborhoods where the area has a greater influence and<br>
there are neighborhoods where the year has a greater influence.<br><br>
You can see how the prediction in green fits the data well.

## Next will be ordinal

In [ ]:
ordinal_cols = {'YrSold', 'GarageCars', 'MoSold', 'KitchenAbvGr', 'BsmtHalfBath', 'BsmtFullBath', 'FullBath', 'HeatingQC', 'BsmtFinType2', 'BsmtExposure', 'BsmtCond', 'BsmtQual', 'ExterCond', 'ExterQual', 'LotShape', 'LandContour', 'LandSlope', 'OverallQual', 'OverallCond'}

df_ordinal = df[['SalePrice', 'Neighborhood', *ordinal_cols]]

In [ ]:
def add_ordinal_rank_column(df: pd.DataFrame, column_name: str, order: list) -> pd.DataFrame:
    df[f'{column_name}_rank'] = df.apply(lambda row: order.index(row[column_name]), axis=1)
    return df.copy()

### understand the values

In [ ]:
for col in ordinal_cols:
    print(col, df_ordinal[col].unique(), sep='\t')

### Fix values into ordinal numbers

In [ ]:
def get_rank(value, order: list) -> int:
    try:
        return order.index(value)
    except:
        return -1

In [ ]:
ordinal_cols_with_order = [
    ('HeatingQC', ['Po', 'Fa', 'TA', 'Gd', 'Ex']),
    ('BsmtFinType2', ['NA', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ']),
    ('BsmtExposure', ['NA', 'No', 'Mn', 'Av', 'Gd']),
    ('BsmtCond', ['Po', 'Fa', 'Ta', 'Gd']),
    ('BsmtQual', ['Fa', 'Ta', 'Gd', 'Ex']),
    ('ExterCond', ['Po', 'Fa', 'TA', 'Gd', 'Ex']),
    ('ExterQual', ['Fa', 'Ta', 'Gd', 'Ex']),
    ('LotShape', ['Reg', 'IR1', 'IR2', 'IR3']),
    ('LandContour', ['Lvl', 'Bnk', 'HLS', 'Low']),
    ('LandSlope', ['Gtl', 'Mod', 'Sev'])
]

In [ ]:
for col_order in ordinal_cols_with_order:
    df_ordinal.insert(0, f'{col_order[0]}_rank',df_ordinal.apply(lambda row: get_rank(row[col_order[0]], col_order[1]), axis=1), True)

## first thing- find correlation between columns and get rid of duplicates

In [ ]:
corr = df_ordinal.corr()
fig, ax = plt.subplots(figsize = (20, 15))
sns.heatmap(corr, annot=True)

### The main findings:
1. OverallQual, ExtraQual_rank -> 0.7
2. OverallQual, BsmtQual_rank -> 0.64
3. LandContour_rank, LandSlope_rank -> 0.61
4. OverallQual, GarageCars -> 0.6
5. OverallQual, FullBath -> 0.55
6. **OverallQual, SalePrice -> 0.79**

#### OverallQual seems to have highest correlation with our target ('SalePrice'), but also be in correlation with the other high corr data
#### Therfore I will stay with it

In [ ]:
# calculate spearman's correlation
corr, _ = spearmanr(df_ordinal.OverallQual, df_ordinal.SalePrice)
print('Spearmans correlation: %.3f' % corr)

df_ordinal.boxplot(column=['SalePrice'], by='OverallQual', grid=False, figsize = (20, 10))

In [ ]:
plt.hist(df['OverallQual'])

In [ ]:
print('The median is: ', df_ordinal.OverallQual.quantile(0.5))
df_ordinal.groupby(['OverallQual'])[['OverallQual']].count()

## So what to do with OverallQual?
Earlier I cleared outliers of area and year compared to the price.<br>
But what makes a value outlier?<br>
This can be due to reasons that are not suitable for learning, for example:<br>
* A very competent real estate agent can increase the selling price.
* An apartment in front of a noisy party club that can reduce the value of the apartment.
* Etc...

### But OverallQual is given!

The median of OverallQual is 6.<br>
**My hypothesis:**<br>
*The higher the score, the more renovated and preserved the house is, which is the opposite of an old house (conceptually).*
<br>

In [ ]:
# this dict keys will be the neighborhoods names and the values will be the corresponding models
area_year_regressors_with_OverallQual = {}

# Note: now we will use the whole DataFrame with "outlairs" to
# test if we can understand them with OverallQual
for nei in df.Neighborhood.unique():
    # only rows of this Neighborhood
    df_temp = df[df.Neighborhood == nei]
    X_train, y_train = df_temp[['GrLivArea', 'YearBuilt', 'OverallQual']].to_numpy(), df_temp.SalePrice.to_numpy()
    
    poly = PolynomialFeatures(2)
    regressor = Ridge(alpha=1000)
    
    X_train_trans = poly.fit_transform(X_train)
    regressor.fit(X_train_trans, y_train)
    
    # add the new calculated regressor to the regressors dict
    area_year_regressors_with_OverallQual[nei] = regressor

In [ ]:
total_improvment = 0
data = []

for nei in df_continuous.Neighborhood.unique():
    # only rows of this Neighborhood
    df_temp = df[df.Neighborhood == nei]
    
    X = df_temp[['GrLivArea', 'YearBuilt']].to_numpy()
    X_trans = poly.fit_transform(X)
    y = df_temp.SalePrice.to_numpy()
    
    regressor = area_year_regressors[nei]
    
    y_predict = regressor.predict(X_trans)
    COUNT_OF_HOUSES= len(y)
#     print(f'{nei} with {COUNT_OF_HOUSES} houses')
    
    # numpy array of each absolute error
    errors = np.absolute(y - y_predict)
    avg_error = errors.sum()/COUNT_OF_HOUSES
    
#     print('avg_error with YearBuilt: ',avg_error)
    
    regressor = area_year_regressors_with_OverallQual[nei]
    
    X = df_temp[['GrLivArea', 'YearBuilt', 'OverallQual']].to_numpy()
    X_trans = poly.fit_transform(X)
    y_predict = regressor.predict(X_trans)
    
    # numpy array of each absolute error
    errors = np.absolute(y - y_predict)
    avg_error2 = errors.sum()/COUNT_OF_HOUSES
    delta = avg_error - avg_error2
#     print('avg_error2 with year_OverallQual: ',avg_error2)
#     print(f'total decrease: {delta} which is {delta/avg_error}%', end='\n\n')
    total_improvment += delta * COUNT_OF_HOUSES
    
    corr, _ = spearmanr(df_temp.OverallQual, df_temp.SalePrice)
    data.append(
        {
            'Neighborhood': nei,
            'COUNT_OF_HOUSES': COUNT_OF_HOUSES,
            'avg_error': avg_error,
            'avg_error_OverallQual': avg_error2,
            'delta': delta,
            'percent': delta/avg_error,
            'corr': corr
        }
    )

print(f'In total the improvment was: {total_improvment}')

In [ ]:
score_df = pd.json_normalize(data).sort_values(by='percent')
score_df

### As you can see, the general prediction improved
Although some of the forecasts for the neighborhoods got worse.

## Categorical (The work on the categorical part is not yet finished)

In [ ]:
categorical_cols = {'GarageType', 'Electrical', 'Heating', 'Foundation', 'CentralAir', 'MasVnrType', 'Exterior2nd', 'Exterior1st', 'RoofMatl', 'RoofStyle', 'MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotConfig', 'Condition1', 'Utilities', 'Condition2', 'BldgType', 'HouseStyle'}
df_cat = df[['SalePrice', 'Neighborhood', *categorical_cols]]

In [ ]:
for cat in categorical_cols:
    df_cat.boxplot(column=['SalePrice'], by=cat, grid=False, figsize = (20, 10))

In [ ]:
# {'Heating', 'Foundation', 'CentralAir', 'Exterior2nd', 'Exterior1st', 'RoofMatl', 'RoofStyle', 'MSSubClass', 'MSZoning', 'Alley', 'Condition1', 'Condition2'}

In [ ]:
# df_cat.groupby('MSZoning')[['MSZoning', 'SalePrice']].median().sort_values(by='SalePrice')

In [ ]:
# GROUPS_NUM = 5
# sub_size = int(df_cat.shape[0]/GROUPS_NUM)-1
# sub_size

In [ ]:
# groups_list = [i for i in range(GROUPS_NUM) for _ in range(sub_size)]
# groups_list.extend((df_cat.shape[0] -len(groups_list)) * [GROUPS_NUM])

# df_cat.sort_values(by='SalePrice', inplace=True)
# df_cat.insert(0, 'price_group', groups_list, True)

In [ ]:
# df.insert(0, 'price_group', df_cat.price_group, True)

In [ ]:
# train_cat = df[:df.shape[0] - sub_size]
# test_cat = df[df.shape[0] - sub_size:]

In [ ]:
# gnb = GaussianNB()
# X_train, y_train = train_cat['CentralAir'], train_cat.price_group
# X_test, y_test = test_cat['CentralAir'], test_cat.price_group

# # accuracies = cross_val_score(gnb, X_train, y_train, cv=5)
# gnb.fit(X_train, y_train)
# y_pred = gnb.predict(X_test)
 

In [ ]:
# def get_category_pred(df: pd.DataFrame, category_col: str) -> pd.DataFrame:
#     t = df.groupby(by=[category_col, 'price_group'])["price_group"].count().reset_index(name="count")
#     s = t.groupby(by=category_col)['count'].idxmax()
#     return t.loc[s]

In [ ]:
# # cat_cols = {'Condition1', 'Condition2', }
# for col in categorical_cols:
#     temp = get_category_pred(df_cat, col)
#     df_cat = df_cat.join(other=temp)

## Submission

In [ ]:
id_list = []
price_list = []

for nei in df_test.Neighborhood.unique():
    # only rows of this Neighborhood
    df_temp = df_test[df_test.Neighborhood == nei]
    regressor = area_year_regressors_with_OverallQual[nei]
    
    X = df_temp[['GrLivArea', 'YearBuilt', 'OverallQual']].to_numpy()
    X_trans = poly.fit_transform(X)
    y_predict = regressor.predict(X_trans)
    
    id_list.extend(df_temp.Id.to_list())
    price_list.extend(y_predict.tolist())

In [ ]:
data = {'Id': id_list, 'SalePrice': price_list}

df_submission = pd.DataFrame(data=data)
df_submission

In [ ]:
df_submission.to_csv('submission.csv', index=False)